In [1]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           47Gi        24Gi       4.9Gi       2.0Mi        18Gi        22Gi
스왑:        16Gi        10Gi       5.2Gi


In [2]:
import os
os.chdir("../function")

In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import load_data
import Beauty_DF
import multiprocessing
from mapping import generate_mapping_function
from mapping_dict import get_breed_dict

In [4]:
dataconn = load_data.LoadData()

ShowDB_Df = dataconn.select_data(
    "show databases"
    )
Schema_all = ShowDB_Df.iloc[:-4,:].values.tolist()
Schema_all = sum(Schema_all, [])

# disconnect from server
dataconn.close()

In [5]:
data = None
start = time.time()
with multiprocessing.Pool(48)  as pool:
    data = list(pool.map(Beauty_DF.makeDf, Schema_all))
end = time.time()
print(end - start)

1505.307344198227


In [6]:
data = pd.concat(data)
display(data)
display(data.isnull().sum())
data.dropna(inplace=True)
data = data.reset_index(drop=True)
display(data)

,BT,Beauty_Count,Price,Name2,Name3,Sex,Day,Label
0,부분미용,1,25000.0,Maltese,몰티즈,3,1972.0,1000
1,전체미용,1,30000.0,Bichon Frise,비숑 프리제,2,NaN,1000
2,전체미용,1,30000.0,Shih Tzu,시추,2,3653.0,1000
3,전체미용,1,30000.0,Shih Tzu,시추,2,3653.0,1000
4,전체미용,1,30000.0,Shih Tzu,시추,2,4018.0,1000
...,...,...,...,...,...,...,...,...
31032,01. 소형견 5킬로 이하,1,35000.0,말티즈 mix,,4,2322.0,989
31033,스포팅추가,1,25000.0,말티즈 mix,,4,2322.0,989
31034,01. 소형견 5킬로 이하,1,35000.0,Maltese,말티즈(몰티즈 마르티즈),4,576.0,989
31035,정방향 클리핑,1,5000.0,Maltese,말티즈(몰티즈 마르티즈),4,576.0,989


BT                   0
Beauty_Count         0
Price                0
Name2                0
Name3                0
Sex                  0
Day             346119
Label                0
dtype: int64

,BT,Beauty_Count,Price,Name2,Name3,Sex,Day,Label
0,부분미용,1,25000.0,Maltese,몰티즈,3,1972.0,1000
1,전체미용,1,30000.0,Shih Tzu,시추,2,3653.0,1000
2,전체미용,1,30000.0,Shih Tzu,시추,2,3653.0,1000
3,전체미용,1,30000.0,Shih Tzu,시추,2,4018.0,1000
4,전체미용,1,30000.0,Shih Tzu,시추,2,4018.0,1000
...,...,...,...,...,...,...,...,...
11299866,01. 소형견 5킬로 이하,1,35000.0,말티즈 mix,,4,2322.0,989
11299867,스포팅추가,1,25000.0,말티즈 mix,,4,2322.0,989
11299868,01. 소형견 5킬로 이하,1,35000.0,Maltese,말티즈(몰티즈 마르티즈),4,576.0,989
11299869,정방향 클리핑,1,5000.0,Maltese,말티즈(몰티즈 마르티즈),4,576.0,989


In [7]:
mapping_dict = get_breed_dict()

tqdm.pandas()
data["Name"] = data.progress_apply(lambda row : generate_mapping_function(row,mapping_dict),axis=1)
data = data[(data["Sex"].isin([1,2,3,4])) & (data["Day"] > 0)]
data = data[data["Name"].isnull() == False]
data = data[data["Name"] != "Unknown"]
data

100%|██████████| 11299871/11299871 [14:06<00:00, 13350.45it/s]


,BT,Beauty_Count,Price,Name2,Name3,Sex,Day,Label,Name
0,부분미용,1,25000.0,Maltese,몰티즈,3,1972.0,1000,Maltese
1,전체미용,1,30000.0,Shih Tzu,시추,2,3653.0,1000,Shih_tzu
2,전체미용,1,30000.0,Shih Tzu,시추,2,3653.0,1000,Shih_tzu
3,전체미용,1,30000.0,Shih Tzu,시추,2,4018.0,1000,Shih_tzu
4,전체미용,1,30000.0,Shih Tzu,시추,2,4018.0,1000,Shih_tzu
...,...,...,...,...,...,...,...,...,...
11299863,스포팅추가,1,30000.0,Bichon Frise,비숑 프리제,4,267.0,989,Bichon_Frise
11299864,특수미용2,1,10000.0,Bichon Frise,비숑 프리제,4,267.0,989,Bichon_Frise
11299868,01. 소형견 5킬로 이하,1,35000.0,Maltese,말티즈(몰티즈 마르티즈),4,576.0,989,Maltese
11299869,정방향 클리핑,1,5000.0,Maltese,말티즈(몰티즈 마르티즈),4,576.0,989,Maltese


In [11]:
data["Beauty_Count"] = data["Beauty_Count"].astype(int)
data2 = data.groupby(["Label"],as_index=False)[["Beauty_Count","Price"]].sum()
data2.to_csv("../File/Beauty.csv",index=False)

221003